# Tests

## Results

Bi-directional LSTM

```
Finished Evaluation [1]: Minibatch[1-1754]: metric = 0.00% * 2000;
```

In [1]:
%load_ext autoreload
%autoreload 1
import cntk as C
import random

In [6]:
import math
import numpy as np
import cntk.tests.test_utils
from cntk import input_variable, cross_entropy_with_softmax, \
        classification_error, sequence
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1)                 # fix a random seed for CNTK componentsk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components

In [7]:
# define dimensions
input_dim =  23897
num_output_classes = 2 
features = sequence.input_variable(shape=input_dim, is_sparse=True)
label = input_variable(num_output_classes)

In [8]:
import sys
import os
from cntk import Trainer, Axis
from cntk.io import MinibatchSource, CTFDeserializer, StreamDef, StreamDefs,\
        INFINITELY_REPEAT
from cntk.learners import adam, sgd, learning_rate_schedule, UnitType, momentum_as_time_constant_schedule
from cntk import input_variable, cross_entropy_with_softmax, \
        classification_error, sequence
from cntk.logging import ProgressPrinter
from cntk.layers import Sequential, Embedding, Recurrence, LSTM, Dense, BatchNormalization

# Creates the reader
def create_reader(path, is_training, input_dim, label_dim):
    """
        Creates a reader that loads data from file.
        
        Args:
            path: is the relative path to the data file.
            is_training: if reader is used for training
            input_dim: the input dim of a word (vocab size)
            label_dim: the number of classes
    """
    return MinibatchSource(CTFDeserializer(path, StreamDefs(
        features=StreamDef(field='S0', shape=input_dim, is_sparse=True),
        labels=StreamDef(field='S1', shape=label_dim, is_sparse=False)
        )), randomize=is_training,
        max_sweeps=INFINITELY_REPEAT if is_training else 1)

def create_criterion_function_preferred(model, labels):
    """
        Creates a tuple of criterions, softmax and classification error
        
        Args:
            model - that is being trained.
            labels - labels to compare with (cntk Variable)
        Returns:
            Tuple of functions: (softmax, classification)
    """
    ce   = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error      (model, labels)
    
    return ce, errs # (model, labels) -> (loss, error metric)

def evaluate(reader, model_func):
    """
        Evaluates the model given a reader (data source) and
        the trained model.
        
        Args:
            reader - data source that reads from files.
            model_func - the trained model.
    """
    # Instantiate the model function; x is the input (feature) variable
    model = model_func(features)

    # Create the loss and error functions
    loss, label_error = create_criterion_function_preferred(model, label)

    # process minibatches and perform evaluation
    progress_printer = C.logging.ProgressPrinter(tag='Evaluation', num_epochs=0)
  
    results = []
    while True:    
        minibatch_size = 500
        data = reader.next_minibatch(minibatch_size, input_map={  # fetch minibatch
            features: reader.streams.features,
            label: reader.streams.labels
        })
        if not data:                                 # until we hit the end
            break

        evaluator = C.eval.Evaluator(loss, progress_printer)
        evaluator.test_minibatch(data)
        results.extend(model.eval(data))
        print data
    evaluator.summarize_test_progress()
    return results

In [9]:
from cntk import load_model
loaded_model = load_model("imbd.v3-1.cntk.mdl")

In [11]:
def do_test():
    """ Tests the network on the validation set.
    """
    input_dim =  23897
    cell_dim = 50
    hidden_dim = 150
    embedding_dim = 100
    num_output_classes = 2 
    
    path = ("data/bbc/aggregated/bbc.init.test.ctf")
    reader = create_reader(path, False, input_dim, num_output_classes)
    return evaluate(reader, loaded_model)
results = do_test()

RuntimeError: Reached the maximum number of allowed errors while reading the input file (data/bbc/aggregated/bbc.init.test.ctf).

[CALL STACK]
    > Microsoft::MSR::CNTK::IDataReader::  SupportsDistributedMBRead
    - Microsoft::MSR::CNTK::IDataReader::  SupportsDistributedMBRead (x5)
    - CreateCompositeDataReader (x3)
    - CNTK::Dictionary::  ~Dictionary (x5)
    - Microsoft::MSR::CNTK::IDataReader::  operator= (x2)



In [35]:
r = np.array(results)
classifications = (np.argmax(r, axis=1))
print classifications

[0 1 1 ..., 0 1 0]
